# Tutorial 05: Vibrational Analysis with Analytical Hessians

In this tutorial, we demonstrate how to use the analytical Hessian capability of `apax`. 
Calculating the Hessian matrix (second derivatives of the energy with respect to atomic positions) is essential for vibrational spectroscopy and thermochemistry.

Traditionally, these are calculated using finite differences of forces, which requires $6N$ model evaluations. `apax` can compute the Hessian analytically using JAX, which is often significantly faster and more precise.

In [ ]:
import warnings

warnings.simplefilter("ignore")

import time
from pathlib import Path

import numpy as np
import yaml
from ase.build import molecule
from ase.optimize import BFGS
from ase.vibrations import Vibrations, VibrationsData

from apax.md.ase_calc import ASECalculator
from apax.utils.datasets import download_etoh_ccsdt, mod_md_datasets
from apax.utils.helpers import mod_config


In [ ]:
!apax template train --full

In [ ]:
# From tutorial 1

data_path = Path("project")

train_file_path, test_file_path = download_etoh_ccsdt(data_path)
train_file_path = mod_md_datasets(train_file_path)
test_file_path = mod_md_datasets(test_file_path)


config_path = Path("config_full.yaml")

config_updates = {
    "n_epochs": 100,
    "data": {
        "n_train": 990,
        "n_valid": 10,
        "valid_batch_size": 10,
        "experiment": "ethanol_ccsd_t",
        "directory": "project/models",
        "data_path": str(train_file_path),
        "test_data_path": str(test_file_path),
        "energy_unit": "kcal/mol",
        "pos_unit": "Ang",
    },
}

config_dict = mod_config(config_path, config_updates)

with open("config_full.yaml", "w") as conf:
    yaml.dump(config_dict, conf, default_flow_style=False)

In [ ]:
!apax train config_full.yaml

In [ ]:
# Note: This tutorial assumes you have a trained model.
# For demonstration, we will use the path to a model directory.
model_dir = "project/models/ethanol_ccsd_t" # Update this to your model path

## 1. Setup and Geometry Optimization

Vibrational analysis must be performed at a stationary point (minimum) on the potential energy surface. First, we'll setup a molecule and relax its geometry.

In [ ]:
atoms = molecule("CH3CH2OH") # Ethanol
calc = ASECalculator(model_dir)
atoms.calc = calc

print("Relaxing geometry...")
opt = BFGS(atoms, logfile=None)
opt.run(fmax=0.001)
print("Geometry relaxed.")

## 2. Analytical Hessian

We can now compute the full Hessian matrix with a single call to `calc.get_hessian(atoms)`.

In [ ]:
start_time = time.time()
hessian_analytical = calc.get_hessian(atoms)
end_time = time.time()

print(f"Analytical Hessian computed in {end_time - start_time:.4f} seconds.")
print(f"Shape: {hessian_analytical.shape}")

## 3. Numerical Hessian (Standard ASE)

For comparison, we perform the same calculation using ASE's numerical finite-difference approach.

In [ ]:
vib = Vibrations(atoms, name='vib_numerical', delta=0.01)

start_time = time.time()
vib.run()
hessian_numerical = vib.get_vibrations().get_hessian().reshape(hessian_analytical.shape)
end_time = time.time()

print(f"Numerical Hessian computed in {end_time - start_time:.4f} seconds.")

## 4. Comparison

Let's check how well the analytical and numerical versions match.

In [ ]:
is_close = np.allclose(hessian_analytical, hessian_numerical, atol=1e-2)
print(f"Hessians match within 1e-2: {is_close}")

# Compare frequencies
vib_data_ana = VibrationsData(atoms, hessian_analytical)
freqs_ana = vib_data_ana.get_frequencies()

vib_data_num = VibrationsData(atoms, hessian_numerical)
freqs_num = vib_data_num.get_frequencies()

print("\nFirst 10 Frequencies (cm^-1):")
print(f"{'Analytical':>15} {'Numerical':>15} {'Diff':>10}")
for i in range(10):
    f_a = freqs_ana[i].real if freqs_ana[i].is_real else freqs_ana[i].imag * 1j
    f_n = freqs_num[i].real if freqs_num[i].is_real else freqs_num[i].imag * 1j
    diff = np.abs(f_a - f_n)
    first = str(f_a)[:14]
    second = str(f_n)[:14]
    print(f"{first:>15} {second:>15} {diff:.4f}\\")